# SLQALCHEMY - PLAIN & SEMI PLAIN SQL

In [30]:
from sqlalchemy import create_engine, Table, MetaData, Column, Integer, String, select, func, text

## CONNECTION

In [31]:
engine = create_engine("sqlite:///db/heroes.db")

In [32]:
connexion = engine.connect()

## QUERY

In [33]:
query = text("""select * from heroes limit 10""")

In [34]:
connexion.execute(query).fetchall()

[(1, 'Crareek'),
 (2, 'Tan'),
 (3, 'Huzzt'),
 (4, 'Pamble'),
 (5, 'Grimm'),
 (6, 'Jarboyd'),
 (7, 'Cyne'),
 (8, 'Wendner'),
 (9, 'Morbid'),
 (10, 'Grrdy')]

## METADATA

In [6]:
metadata = MetaData(bind=connexion)

> METADATA is used to store database object info/data, such as schema

## TABLE

- To get Column arguments such as nullable, unique, primary_key using sqlite vscode extension, connect to the db, and in sqlite explorer, hover the column to get them

- Otherwise use sql browser

In [7]:
heroes_table = Table('heroes', metadata, 
                     Column('hero_id', Integer, nullable=False, primary_key=True),
                     Column('name', String(20), unique=True, nullable=False),
                     extend_existing= True)

## WHERE , ORDER BY

In [8]:
query = (select([heroes_table])
            .where(heroes_table.c.name.startswith("C"))
            .order_by(heroes_table.c.name.asc()))
connexion.execute(query).fetchall()

[(109, 'Chuz'),
 (42, 'Cinder'),
 (56, 'Cookerry'),
 (71, 'Cookie'),
 (38, 'Crane'),
 (1, 'Crareek'),
 (69, 'Crystal'),
 (7, 'Cyne')]

## AND, OR operator

In [9]:
query = (select([heroes_table])
            .where(heroes_table.c.name.startswith("C") & (heroes_table.c.hero_id < 50) )
            .order_by(heroes_table.c.hero_id))
connexion.execute(query).fetchall()

[(1, 'Crareek'), (7, 'Cyne'), (38, 'Crane'), (42, 'Cinder')]

In [10]:
query = (select([heroes_table])
            .where(heroes_table.c.name.startswith("C") | (heroes_table.c.name.startswith("P")) )
            .order_by(heroes_table.c.name))
connexion.execute(query).fetchall()

[(109, 'Chuz'),
 (42, 'Cinder'),
 (56, 'Cookerry'),
 (71, 'Cookie'),
 (38, 'Crane'),
 (1, 'Crareek'),
 (69, 'Crystal'),
 (7, 'Cyne'),
 (70, 'Pador'),
 (4, 'Pamble'),
 (28, 'Perabiel'),
 (32, 'Pierceslight'),
 (41, 'Pontihill')]

## INSERT

### INSERT SINGLE VALUE USING PLAIN TEXT

> sqlachemy request text for plain text query, as deprecation notice effective in coming 2.0 release

In [54]:
value = {"name": "Toto"}
query = text("insert into heroes (name) values (:name)")

In [17]:
connexion.execute(query, value)

In [104]:
query = text("select * from heroes where name = (:name)")
connexion.execute(query, value).fetchall()

[(123, 'Toto')]

### INSERT MULTIPLE VALUES USING PLAIN TEXT QUERY

In [56]:
values = [{"name": "Titi"}, {"name": "Nemo"}]
query = text("insert into heroes (name) values (:name)")

In [21]:
connexion.execute(query, values)

#### VERIFY INSERTED VALUES

In [58]:
txt_in = ', '.join(["'{0}'".format(v['name']) for v in [*values, value]])
txt = f"select * from heroes where name in ({txt_in})"
print(txt)

select * from heroes where name in ('Titi', 'Nemo', 'Toto')


> NB: it is important using plain text query to surround values with quotes

In [60]:
verify_query = text(txt)

In [61]:
connexion.execute(verify_query).fetchall()

[(125, 'Nemo'), (124, 'Titi'), (123, 'Toto')]

## DELETE

### DELETE MULTIPLE VALUES USING PLAIN TEXT QUERY

In [62]:
values = [{"name": "Titi"}, {"name": "Toto"}, {"name": "Nemo"}]
query = text("delete from heroes where name = (:name)")

In [63]:
connexion.execute(query, values)

#### VERIFY DELETED DATA

In [64]:
verify_query = text("select * from heroes where name = (:name)")
assert not connexion.execute(verify_query, values).fetchall()

## AGGREGATE DATA

### Loop over results

In [24]:
query = """select * from heroes limit 10"""
results = connexion.execute(query)

In [22]:
for row in results:
    print(row)

(1, 'Crareek')
(2, 'Tan')
(3, 'Huzzt')
(4, 'Pamble')
(5, 'Grimm')
(6, 'Jarboyd')
(7, 'Cyne')
(8, 'Wendner')
(9, 'Morbid')
(10, 'Grrdy')


In [58]:
battle_events_table = Table('battle_events', metadata,
                            Column('battle_event_id', Integer, primary_key=True, nullable=False),
                            Column('battle_participant_id', Integer, nullable=False),
                            Column('battle_event_type_id', Integer, nullable=False),
                            Column('rubies_gained', Integer),
                            Column('timestamp', Integer, nullable=False)
                            )

### FUNC

#### FUNC SUM

In [63]:
query = select([func.sum(battle_events_table.c.rubies_gained)])
result = connexion.execute(query).scalar()
result

72847

#### FUNC MAX

In [65]:
query = select([func.max(battle_events_table.c.rubies_gained)])
result = connexion.execute(query)
result.scalar()

15